In [1]:
# Import Dependencies
import pandas as pd
import numpy as np

In [2]:
# Files to Load
clean_hawaii_measurements_load = "clean_hawaii_measurements.csv"
clean_hawaii_stations_load = "clean_hawaii_stations.csv"

# Read measurements and stations Data File and store into Pandas Data Frames
hawaii_measurements_df = pd.read_csv(clean_hawaii_measurements_load)
hawaii_stations_df = pd.read_csv(clean_hawaii_stations_load)

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, DateTime
from sqlalchemy.orm import Session

In [4]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite", echo=True)

# Create a connection to the engine called `conn`
conn = engine.connect()

2018-04-03 22:19:47,999 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-04-03 22:19:48,002 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:19:48,006 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-04-03 22:19:48,011 INFO sqlalchemy.engine.base.Engine ()


In [5]:
# Use `declarative_base` from SQLAlchemy to model the demographics table as an ORM class
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'

    station = Column(Text, primary_key=True)
    date = Column(Text, primary_key=True)
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Station(Base):
    __tablename__ = 'station'

    station = Column(Text, primary_key=True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

# Use `create_all` to create the tables in the database
Base.metadata.create_all(engine)

session = Session(bind=engine)

2018-04-03 22:19:53,123 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-04-03 22:19:53,127 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:19:53,132 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("station")
2018-04-03 22:19:53,134 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:19:53,139 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE measurement (
	station TEXT NOT NULL, 
	date TEXT NOT NULL, 
	prcp FLOAT, 
	tobs INTEGER, 
	PRIMARY KEY (station, date)
)


2018-04-03 22:19:53,142 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:19:53,147 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-03 22:19:53,150 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE station (
	station TEXT NOT NULL, 
	name TEXT, 
	latitude FLOAT, 
	longitude FLOAT, 
	elevation FLOAT, 
	PRIMARY KEY (station)
)


2018-04-03 22:19:53,176 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:19:53,184 INFO sqlalchemy.engine.base.Engine COMMIT


In [6]:
# Use Orient='records' to create a list of data to write
measurement_data = hawaii_measurements_df.to_dict(orient='records')
measurement_data[0]

stations_data = hawaii_stations_df.to_dict(orient='records')
stations_data[0]

{'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [7]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

2018-04-03 22:20:19,942 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2018-04-03 22:20:19,946 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:20:19,951 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("measurement")
2018-04-03 22:20:19,954 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:20:19,959 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2018-04-03 22:20:19,962 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:20:19,965 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("measurement")
2018-04-03 22:20:19,968 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:20:19,970 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'measurement' AND type = 'table'
2018-04-03 22:20:19,973 INFO sqlalchemy.engin

In [8]:
# Save the reference to the `measurement` table as a variable called `table`
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)
station_table = sqlalchemy.Table('station', metadata, autoload=True)

In [13]:
# Use `table.insert()` to insert the data into the table
conn.execute(measurement_table.insert(), measurement_data)
conn.execute(station_table.insert(), stations_data)

2018-04-03 22:26:08,322 INFO sqlalchemy.engine.base.Engine INSERT INTO measurement (station, date, prcp, tobs) VALUES (?, ?, ?, ?)
2018-04-03 22:26:08,326 INFO sqlalchemy.engine.base.Engine (('USC00519397', '2010-01-01', 0.08, 65), ('USC00519397', '2010-01-02', 0.0, 63), ('USC00519397', '2010-01-03', 0.0, 74), ('USC00519397', '2010-01-04', 0.0, 76), ('USC00519397', '2010-01-06', 0.0, 73), ('USC00519397', '2010-01-07', 0.06, 70), ('USC00519397', '2010-01-08', 0.0, 64), ('USC00519397', '2010-01-09', 0.0, 68)  ... displaying 10 of 19550 total bound parameter sets ...  ('USC00516128', '2017-08-22', 0.5, 76), ('USC00516128', '2017-08-23', 0.45, 76))
2018-04-03 22:26:08,466 INFO sqlalchemy.engine.base.Engine COMMIT
2018-04-03 22:26:08,491 INFO sqlalchemy.engine.base.Engine INSERT INTO station (station, name, latitude, longitude, elevation) VALUES (?, ?, ?, ?, ?)
2018-04-03 22:26:08,502 INFO sqlalchemy.engine.base.Engine (('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0), ('USC

In [15]:
conn.execute("select * from measurement limit 5").fetchall()
conn.execute("select * from station limit 5").fetchall()

2018-04-03 22:28:53,157 INFO sqlalchemy.engine.base.Engine select * from measurement limit 5
2018-04-03 22:28:53,161 INFO sqlalchemy.engine.base.Engine ()
2018-04-03 22:28:53,165 INFO sqlalchemy.engine.base.Engine select * from station limit 5
2018-04-03 22:28:53,167 INFO sqlalchemy.engine.base.Engine ()


[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]